In [1]:
import json
import re
import sqlite3

# Data Structure for AST Node
class Node:
    def __init__(self, node_type, value=None):
        self.type = node_type  # "operator" or "operand"
        self.left = None       # Left child
        self.right = None      # Right child
        self.value = value     # Value for operands (e.g., "age > 30")

    def __repr__(self):
        if self.type == "operator":
            return f"({self.left} {self.value} {self.right})"
        else:
            return str(self.value)

# Initialize SQLite database connection
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()

# Create Rules table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Rules (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rule_string TEXT NOT NULL,
    ast_json TEXT NOT NULL
)
''')
conn.commit()

# Function to create a rule and return its AST
def create_rule(rule_string):
    tokens = re.split(r'(\s+AND\s+|\s+OR\s+|\(|\))', rule_string)
    tokens = [token.strip() for token in tokens if token.strip()]

    def parse_expression(index):
        stack = []
        while index < len(tokens):
            token = tokens[index]
            if token == '(':
                sub_expression, index = parse_expression(index + 1)
                stack.append(sub_expression)
            elif token == ')':
                break
            elif token in ('AND', 'OR'):
                stack.append(token)
            else:
                # It's a condition
                condition_node = parse_condition(token)
                stack.append(condition_node)
            index += 1

        return combine_conditions(stack), index

    def combine_conditions(stack):
        if not stack:
            return None

        left = stack.pop(0)

        while stack:
            operator = stack.pop(0)
            right = stack.pop(0) if stack else None

            if right is not None:
                node = Node("operator", operator)
                node.left = left
                node.right = right
                left = node
        return left

    def parse_condition(condition_str):
        match = re.match(r'(\w+)\s*(>|<|=)\s*([\'"]?\w+[\'"]?)', condition_str.strip())
        if match:
            return Node("operand", f"{match.group(1)} {match.group(2)} {match.group(3)}")
        return None

    ast, _ = parse_expression(0)
    return ast

# Function to combine multiple rules into a single AST
def combine_rules(rules):
    combined_stack = []
    for rule in rules:
        combined_stack.append(create_rule(rule))

    # Create a combined rule based on the first rule as a starting point
    combined_rule = combined_stack[0]
    for rule_ast in combined_stack[1:]:
        # Combine using OR for simplicity, can be improved based on needs
        combined = Node("operator", "OR")
        combined.left = combined_rule
        combined.right = rule_ast
        combined_rule = combined

    return combined_rule

# Function to evaluate the AST against provided data
def evaluate_operand(node, data):
    field, operator, value = node.value.split()
    value = value.strip("'\"")  # Remove quotes from string values

    # Convert numeric fields
    if field in data:
        data_value = data[field]
        if isinstance(data_value, str) and value.isdigit():
            data_value = int(data_value)

        if operator == ">":
            return data_value > int(value)
        elif operator == "<":
            return data_value < int(value)
        elif operator == "=":
            return data_value == value
    return False

def evaluate_rule(ast, data):
    if ast is None:
        return False  # Return False if AST is None

    if ast.type == "operand":
        return evaluate_operand(ast, data)
    elif ast.type == "operator":
        left_result = evaluate_rule(ast.left, data)
        right_result = evaluate_rule(ast.right, data)

        if ast.value == "AND":
            return left_result and right_result
        elif ast.value == "OR":
            return left_result or right_result

    return False

# Function to modify a rule
def modify_rule(ast, field, new_value, operator):
    if ast is None:
        return

    if ast.type == "operand":
        if field in ast.value:
            ast.value = f"{field} {operator} {new_value}"

    else:
        modify_rule(ast.left, field, new_value, operator)
        modify_rule(ast.right, field, new_value, operator)

# Function to validate rule strings
def validate_rule_string(rule_string):
    # A better regex to validate rule strings
    regex = r'^\s*(\(\s*|\s*(\w+\s*(>|<|=)\s*([\'"]?\w+[\'"]?)\s*|\s*(AND|OR)\s*)*|\s*\))$'
    if not re.match(regex, rule_string.strip()):
        raise ValueError("Invalid rule format")

# Function to store a rule in the database
def store_rule_in_db(rule_string, ast):
    ast_json = json.dumps(ast, default=lambda o: o.__dict__)  # Serialize AST to JSON
    cursor.execute('''
    INSERT INTO Rules (rule_string, ast_json)
    VALUES (?, ?)
    ''', (rule_string, ast_json))
    conn.commit()

# Function to retrieve all stored rules
def retrieve_rules_from_db():
    cursor.execute('SELECT * FROM Rules')
    rows = cursor.fetchall()
    return rows

# Test Cases
# Test Case 1: Create individual rules and verify their AST
rule1 = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
ast1 = create_rule(rule1)
print("AST for Rule 1:", ast1)

# Test Case 2: Combine rules
rule2 = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"
combined_ast = combine_rules([rule1, rule2])
print("Combined AST:", combined_ast)

# Test Case 3: Evaluate rule
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
print("Evaluation Result for Rule 1:", evaluate_rule(ast1, data))  # Expected: True

# Test Case 4: Modify rule
modify_rule(ast1, 'salary', 70000, '>')
print("Modified Evaluation Result for Rule 1:", evaluate_rule(ast1, data))  # Expected: False after modification

# Test Case 5: Error handling
try:
    invalid_rule = "age > 30 AND"
    validate_rule_string(invalid_rule)
except ValueError as e:
    print("Error:", e)

# Bonus: Storing and retrieving rules
store_rule_in_db(rule1, ast1)
rules = retrieve_rules_from_db()
print("Stored Rules:", rules)

# Close the database connection
conn.close()


AST for Rule 1: (((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5))
Combined AST: ((((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)) OR ((age > 30 AND department = 'Marketing') AND (salary > 20000 OR experience > 5)))
Evaluation Result for Rule 1: True
Modified Evaluation Result for Rule 1: False
Stored Rules: [(1, "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)", '{"type": "operator", "left": {"type": "operator", "left": {"type": "operator", "left": {"type": "operand", "left": null, "right": null, "value": "age > 30"}, "right": {"type": "operand", "left": null, "right": null, "value": "department = \'Sales\'"}, "value": "AND"}, "right": {"type": "operator", "left": {"type": "operand", "left": null, "right": null, "value": "age < 25"}, "right": {"type": "operand", "left": nu